# Gathering Data from https://www.altafsir.com/

In [64]:
import pandas as pd
from selenium import webdriver
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
from unidecode import unidecode


In [65]:

def convert_to_english(name):
    # Remove diacritics and non-alphabetic characters
    name = unidecode(name)
    name = re.sub(r'[^a-zA-Z\s]+', '', name)
    return name

In [66]:
service = webdriver.ChromeService(executable_path = './chromedriver-mac-arm64/chromedriver')
driver = webdriver.Chrome(service=service)

In [67]:
url = 'https://thaqalayn.net/'
driver.get(url)

In [68]:
chrome_options = Options()
prefs = {
    "profile.default_content_settings": {"automatic_downloads": 1},
    "profile.managed_default_content_settings": {"images": 2}
}
chrome_options.add_experimental_option("prefs", prefs)

# Go to the end with loop now if you want to run it all

In [69]:
url = "https://thaqalayn.net/book/" + "7"
driver.get(url)
try:
    if "404" in driver.find_element(By.CLASS_NAME, "next-error-h1").text:
        print("Data Not Found")
except:
    print("Data Found on URL")


Data Found on URL


In [70]:
title = driver.find_element(By.TAG_NAME, "h1").text
title

'Al-Kāfi'

In [71]:
title_en = driver.find_element(By.CLASS_NAME, "text-2xl").text
title_en

'The Sufficient'

In [72]:
volume = driver.find_elements(By.CLASS_NAME, "nextui-button-text")

if len(volume) > 1:
    volume = volume[1].text
else:
    volume = ""
volume

'Volume 7'

In [73]:
author = driver.find_elements(By.TAG_NAME, "h4")[0].text.split(": ")[-1]
author

'Shaykh Muḥammad b. Yaʿqūb al-Kulaynī'

In [74]:
translator = driver.find_elements(By.TAG_NAME, "h4")[1].text.split(": ")[-1]
translator

'Shaykh Muhammad Sarwar'

In [ ]:
description = driver.find_element(By.TAG_NAME, "h5").text
description

In [76]:
books = []
for i in driver.find_elements(By.CLASS_NAME, "m-auto"):
    if "|" in i.text:
        books.append(i.text)

In [77]:
len(books)

7

In [78]:
books[0]

'1 | The Book of Wills'

In [79]:
chapter_list = []
for i in driver.find_elements(By.PARTIAL_LINK_TEXT, "Chapter"):
    text = i.text.split("\n")
    text = text[0] + " | " + text[2]
    chapter_list.append(text)

In [80]:

# Initialize a dictionary to store grouped chapters
grouped_chapters = []
current_chapter_group = []

# Iterate through the list
for item in chapter_list:
    if item.startswith("Chapter 1 "):
        if current_chapter_group:
            # If there is a current chapter group, append it to the grouped chapters
            grouped_chapters.append(current_chapter_group)
            current_chapter_group = []  # Reset current chapter group
    current_chapter_group.append(item)

# Append the last chapter group if any
if current_chapter_group:
    grouped_chapters.append(current_chapter_group)


In [81]:
len(grouped_chapters)

5

In [ ]:
if len(grouped_chapters) != len(books):
    print("The last chapter from book '0 | Introduction' is hidden! Manually click it for now")
    time.sleep(5)
    chapter_list = []
    for i in driver.find_elements(By.PARTIAL_LINK_TEXT, "Chapter"):
        text = i.text.split("\n")
        text = text[0] + " | " + text[2]
        chapter_list.append(text)
    
    # Initialize a dictionary to store grouped chapters
    grouped_chapters = []
    current_chapter_group = []

    # Iterate through the list
    for item in chapter_list:
        if item.startswith("Chapter 1 "):
            if current_chapter_group:
                # If there is a current chapter group, append it to the grouped chapters
                grouped_chapters.append(current_chapter_group)
                current_chapter_group = []  # Reset current chapter group
        current_chapter_group.append(item)

    # Append the last chapter group if any
    if current_chapter_group:
        grouped_chapters.append(current_chapter_group)

len(grouped_chapters)

In [ ]:
books

In [ ]:
chapter_list

In [ ]:
hadees = []
for i in range(len(books)):
    pattern = r'^(\d+)'
    match = re.search(pattern, books[i])
    book_index = match.group(1)
    hadees.append([])
    for j in range(len(grouped_chapters[i])):
        pattern = r'^Chapter\s*(\d+)'
        match = re.search(pattern, grouped_chapters[i][j])
        chapter_index = match.group(1)

        url = "https://thaqalayn.net/chapter/1/" + book_index + "/" + chapter_index
        driver.get(url)
        hadees[i].append([])

        for k in driver.find_elements(By.CLASS_NAME, "nextui-c-BDLTQ"):
            text = k.text.split("\n")
            if "Thaqalayn" not in text[0]:
                text = {"hadees_number": text[0], "hadees_arabic": text[2], "hadees_english": text[3]}
                hadees[i][j].append(text)

        #print(book_index, chapter_index)

In [ ]:
hadees[0][0][0]

{'number': 'Ḥadīth #1',
 'arabic': '1ـ أَخْبَرَنَا أَبُو جَعْفَرٍ مُحَمَّدُ بْنُ يَعْقُوبَ قَالَ حَدَّثَنِي عِدَّةٌ مِنْ أَصْحَابِنَا مِنْهُمْ مُحَمَّدُ بْنُ يَحْيَى الْعَطَّارُ عَنْ أَحْمَدَ بْنِ مُحَمَّدٍ عَنِ الْحَسَنِ بْنِ مَحْبُوبٍ عَنِ الْعَلاءِ بْنِ رَزِينٍ عَنْ مُحَمَّدِ بْنِ مُسْلِمٍ عَنْ ابي جعفر قَالَ لَمَّا خَلَقَ الله الْعَقْلَ اسْتَنْطَقَهُ ثُمَّ قَالَ لَهُ أَقْبِلْ فَأَقْبَلَ ثُمَّ قَالَ لَهُ أَدْبِرْ فَأَدْبَرَ ثُمَّ قَالَ وَعِزَّتِي وَجَلالِي مَا خَلَقْتُ خَلْقاً هُوَ أَحَبُّ إِلَيَّ مِنْكَ وَلا أَكْمَلْتُكَ إِلا فِيمَنْ أُحِبُّ أَمَا إِنِّي إِيَّاكَ آمُرُ وَإِيَّاكَ أَنْهَى وَإِيَّاكَ أُعَاقِبُ وَإِيَّاكَ أُثِيبُ.',
 'english': "1. Abu Ja’far Muhammad b. Ya’qub (al-Kulayni) has said: A number of our people of whom one is Muhammad b. Yahya al-‘Attar narrated from Ahmad b. Muhammad from Hasan b. Mahbub from al-’Ala b. Razin from Muhammad b. Muslim that Abu Ja’far (al-Baqir) has said: “When Allah (swt) created the intellect (al-'Aql) He made it speak and said to it, ‘Com

In [ ]:
flat_data = []
for i in range(len(books)):
    book_name = books[i]
    for j in range(len(grouped_chapters[i])):
        chapter_name = grouped_chapters[i][j]
        for k in range(len(hadees[i][j])):
            hadees_current = hadees[i][j][k]
            flat_data.append({
                'title': title,
                'title_en': title_en,
                'volume': volume,
                'author': author,
                'translator': translator,
                'description': description,
                'book': book_name,
                'chapter': chapter_name,
                'hadees_number': hadees_current['number'],
                'hadees_arabic': hadees_current['arabic'],
                'hadees_english': hadees_current['english']
            })


In [ ]:
df = pd.DataFrame(flat_data)

# Display DataFrame
df.head()

,title,category,volume,author,translator,description,book,chapter,hadees_number,hadees_arabic,hadees_english
0,Al-Kāfi,The Sufficient,Volume 1,Shaykh Muḥammad b. Yaʿqūb al-Kulaynī,Shaykh Muhammad Sarwar,"al-Kāfī, compiled by Shaykh al-Kulaynī, is a m...",1 | The Book of Intelligence and Ignorance,Chapter 0 | The Book of Intelligence and Ignor...,Ḥadīth #1,1ـ أَخْبَرَنَا أَبُو جَعْفَرٍ مُحَمَّدُ بْنُ ي...,1. Abu Ja’far Muhammad b. Ya’qub (al-Kulayni) ...
1,Al-Kāfi,The Sufficient,Volume 1,Shaykh Muḥammad b. Yaʿqūb al-Kulaynī,Shaykh Muhammad Sarwar,"al-Kāfī, compiled by Shaykh al-Kulaynī, is a m...",1 | The Book of Intelligence and Ignorance,Chapter 0 | The Book of Intelligence and Ignor...,Ḥadīth #2,2ـ عَلِيُّ بْنُ مُحَمَّدٍ عَنْ سَهْلِ بْنِ زِي...,2. Ali bin Muhammad has narrated from Sahl ibn...
2,Al-Kāfi,The Sufficient,Volume 1,Shaykh Muḥammad b. Yaʿqūb al-Kulaynī,Shaykh Muhammad Sarwar,"al-Kāfī, compiled by Shaykh al-Kulaynī, is a m...",1 | The Book of Intelligence and Ignorance,Chapter 0 | The Book of Intelligence and Ignor...,Ḥadīth #3,3ـ أَحْمَدُ بْنُ إِدْرِيسَ عَنْ مُحَمَّدِ بْنِ...,3. Ahmad ibn ’Idris has narrated from Muhammad...
3,Al-Kāfi,The Sufficient,Volume 1,Shaykh Muḥammad b. Yaʿqūb al-Kulaynī,Shaykh Muhammad Sarwar,"al-Kāfī, compiled by Shaykh al-Kulaynī, is a m...",1 | The Book of Intelligence and Ignorance,Chapter 0 | The Book of Intelligence and Ignor...,Ḥadīth #4,4ـ مُحَمَّدُ بْنُ يَحْيَى عَنْ أَحْمَدَ بْنِ م...,4. Muhammad ibn Yahya has narrated from Ahmad ...
4,Al-Kāfi,The Sufficient,Volume 1,Shaykh Muḥammad b. Yaʿqūb al-Kulaynī,Shaykh Muhammad Sarwar,"al-Kāfī, compiled by Shaykh al-Kulaynī, is a m...",1 | The Book of Intelligence and Ignorance,Chapter 0 | The Book of Intelligence and Ignor...,Ḥadīth #5,5ـ وَعَنْهُ عَنْ أَحْمَدَ بْنِ مُحَمَّدٍ عَنِ ...,5. It is narrated from him from Ahmad ibn Muha...


In [ ]:
file_name = convert_to_english(title)+"_"+convert_to_english(title_en)+"_"+volume+"_"+convert_to_english(author)+".csv"
file_name = file_name.replace(" ","")
df.to_csv(file_name, index=False)

# NOW WE MAKE A LOOP FOR THE WHOLE WEBSITE!

In [95]:
for main_ind in range(11, 35):
    driver.get("https://thaqalayn.net/book/" + str(main_ind))
    try:
        if "404" in driver.find_element(By.CLASS_NAME, "next-error-h1").text:
            print("Data Not Found")
            continue
    except:
        print("Data Found on URL")

    title = driver.find_element(By.TAG_NAME, "h1").text
    title_en = driver.find_element(By.CLASS_NAME, "text-2xl").text
    volume = driver.find_elements(By.CLASS_NAME, "nextui-button-text")
    if len(volume) > 1:
        volume = volume[1].text
    else:
        volume = ""
    author = driver.find_elements(By.TAG_NAME, "h4")[0].text.split(": ")[-1]
    translator = driver.find_elements(By.TAG_NAME, "h4")[1].text.split(": ")[-1]

    try:
        description = driver.find_element(By.TAG_NAME, "h5").text
    except:
        description = ""
    books = []
    for i in driver.find_elements(By.CLASS_NAME, "m-auto"):
        if "|" in i.text or "Section" in i.text:
            books.append(i.text)

    chapter_list = []
    for i in driver.find_elements(By.PARTIAL_LINK_TEXT, "Chapter"):
        text = i.text.split("\n")
        try:
            text = text[0] + " | " + text[2]
            chapter_list.append(text)
        except:
            pass
    grouped_chapters = []
    current_chapter_group = []
    for item in chapter_list:
        if item.startswith("Chapter 1 "):
            if current_chapter_group:
                grouped_chapters.append(current_chapter_group)
                current_chapter_group = []
        current_chapter_group.append(item)

    if current_chapter_group:
        grouped_chapters.append(current_chapter_group)

    if len(grouped_chapters) != len(books):
        print("The last chapter from book '0 | Introduction' is hidden! Manually click it for now")
        time.sleep(5)
        chapter_list = []
        for i in driver.find_elements(By.PARTIAL_LINK_TEXT, "Chapter"):
            text = i.text.split("\n")
            print(text)
            try:
                text = text[0] + " | " + text[2]
                #if "Chapter 66 | Another Chapter" in text: #solving a problem for https://thaqalayn.net/book/7 where book 3&4 are not visible
                #    chapter_list.append("Chapter 1 | -")
                #    chapter_list.append("Chapter 1 | -")
                chapter_list.append(text)
            except:
                pass
        
        # Initialize a dictionary to store grouped chapters
        grouped_chapters = []
        current_chapter_group = []

        # Iterate through the list
        for item in chapter_list:
            if item.startswith("Chapter 1 "):
                if current_chapter_group:
                    # If there is a current chapter group, append it to the grouped chapters
                    grouped_chapters.append(current_chapter_group)
                    current_chapter_group = []  # Reset current chapter group
            current_chapter_group.append(item)

        # Append the last chapter group if any
        if current_chapter_group:
            grouped_chapters.append(current_chapter_group)

    
    hadees = []
    for i in range(len(books)):
        pattern = r'(\d+)'
        match = re.search(pattern, books[i])
        book_index = match.group(1)
        hadees.append([])
        for j in range(len(grouped_chapters[i])):
            pattern = r'^Chapter\s*(\d+)'
            match = re.search(pattern, grouped_chapters[i][j])
            chapter_index = match.group(1)

            url = "https://thaqalayn.net/chapter/" + str(main_ind) + "/" + book_index + "/" + chapter_index
            driver.get(url)
            hadees[i].append([])

            for k in driver.find_elements(By.CLASS_NAME, "nextui-c-BDLTQ"):
                text = k.text.split("\n")
                if "Thaqalayn" not in text[0]:
                    while len(text) < 4:
                        text.append("")
                    text = {"hadees_number": text[0], "hadees_arabic": text[2], "hadees_english": text[3]}
                    hadees[i][j].append(text)

    
    flat_data = []
    for i in range(len(books)):
        book_name = books[i]
        for j in range(len(grouped_chapters[i])):
            chapter_name = grouped_chapters[i][j]
            for k in range(len(hadees[i][j])):
                hadees_current = hadees[i][j][k]
                flat_data.append({
                    'title': title,
                    'title_en': title_en,
                    'volume': volume,
                    'author': author,
                    'translator': translator,
                    'description': description,
                    'book': book_name,
                    'chapter': chapter_name,
                    'hadees_number': hadees_current['hadees_number'],
                    'hadees_arabic': hadees_current['hadees_arabic'],
                    'hadees_english': hadees_current['hadees_english']
                })

    
    df = pd.DataFrame(flat_data)

    file_name = convert_to_english(title)+"_"+convert_to_english(title_en)+"_"+volume+"_"+convert_to_english(author)+".csv"
    file_name = file_name.replace(" ","")
    df.to_csv(file_name, index=False)

    
    

Data Found on URL
Data Found on URL
Data Found on URL
Data Found on URL
Data Not Found
Data Not Found
Data Found on URL
Data Not Found
Data Not Found
Data Not Found
Data Not Found
Data Found on URL
Data Found on URL
Data Found on URL
Data Found on URL
Data Found on URL
Data Found on URL
Data Found on URL
Data Found on URL
Data Found on URL
Data Found on URL
Data Found on URL
Data Found on URL
Data Not Found


### Almost took 200 minutes to run this loop!